In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-02-29"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
13320,2024-02-29,Rússia Superliga,13:00,Tambov,Lokomotiv Kuban 2,1.30,3.31,156.5,1.80,1.90,0.0,0.0,0.0,xQVFUbDQ,0.769231,0.302115,0.555556,0.526316,0.071346,228.912,142.051572,0.620551,0.6,1.341641,2.236068,392.60,3.300,2.383820,0.722370,-59.0,266.314,184.628508,0.693274,1.2,1.643168,1.369306,184.14,3.612,2.427812,0.672152,-15.0,65,93,6.04,1.98,175.482,400.972,0.616609,0.038222,NaN,-3.61,-0.722,-0.415512,0.000000,0.0,0.000000,2.80,0.560,4.125000,0.000000,0.0,0.000000
13321,2024-02-29,Romênia Liga Nacional Feminina,14:00,Sepsi S. F,CSM Constanta F,1.25,3.69,151.5,1.88,1.78,0.0,0.0,0.0,ljGUMcSi,0.800000,0.271003,0.531915,0.561798,0.071003,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,189.372,201.616900,1.064661,3.0,0.000000,0.000000,548.02,2.158,2.069860,0.959157,124.0,135,94,1.02,5.83,0.000,209.002,0.698518,0.038640,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,-0.84,-0.168,-16.011905,0.000000,0.0,0.000000
13322,2024-02-29,Rússia Liga Vtb United,11:00,Uralmash Ekaterinburg,Nizhny Novgorod,1.38,3.08,153.5,1.82,1.94,0.0,0.0,0.0,lzz83bji,0.724638,0.324675,0.549451,0.515464,0.049313,140.230,42.960802,0.306360,2.4,1.341641,0.559017,96.12,1.682,0.653429,0.388484,33.0,346.632,277.262223,0.799875,1.2,1.643168,1.369306,76.50,4.382,2.583345,0.589536,-37.0,89,50,1.08,1.53,151.296,311.338,0.539050,0.045134,NaN,0.91,0.182,2.087912,0.000000,0.0,0.000000,-5.00,-1.000,-2.080000,0.000000,0.0,0.000000
13323,2024-02-29,Rússia Superliga,06:30,Dynamo Vladivostok,University-Ugra,1.22,3.98,154.5,1.83,1.87,0.0,0.0,0.0,UaS7WKrE,0.819672,0.251256,0.546448,0.534759,0.070928,117.178,15.449528,0.131847,2.4,1.341641,0.559017,112.20,1.404,0.261209,0.186046,70.0,176.898,124.716664,0.705020,0.6,1.341641,2.236068,162.54,2.388,1.882756,0.788424,-23.0,102,86,1.10,1.89,0.000,185.740,0.750621,0.015289,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,-3.11,-0.622,-4.790997,0.000000,0.0,0.000000
13324,2024-02-29,Rússia Superliga,12:00,Zenit Petersburg 2,Dome-Springs Izhevsk,1.60,2.24,152.5,1.85,1.85,0.0,0.0,0.0,z1BjOdjf,0.625000,0.446429,0.540541,0.540541,0.071429,214.182,65.292846,0.304847,2.4,1.341641,0.559017,245.41,2.588,0.777895,0.300578,46.0,227.878,180.226566,0.790891,2.4,1.341641,0.559017,120.70,3.124,2.980299,0.954001,38.0,97,85,2.53,1.42,195.744,0.000,0.235702,0.000000,NaN,2.17,0.434,1.382488,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13413,2024-02-29,Eua Ncaa,20:30,Ohio State,Nebraska,1.73,2.10,143.5,1.80,1.91,0.0,0.0,0.0,MZQn7CFs,0.578035,0.476190,0.555556,0.523560,0.054225,217.872,59.021706,0.270901,1.8,1.643168,0.912871,265.20,3.308,1.258956,0.380579,-6.0,132.430,31.797037,0.240104,2.4,1.341641,0.559017,100.01,1.730,0.452769,0.261716,49.0,60,73,4.42,1.37,154.050,161.616,0.136621,0.041931,NaN,1.66,0.332,2.198795,0.707148,0.7,-0.007148,-3.17,-0.634,-1.735016,0.000000,0.0,0.000000
13414,2024-02-29,Eua Ncaa,20:07,Binghamton,Maine,1.64,2.23,131.5,1.80,1.91,0.0,0.0,0.0,ttX0hJ4t,0.609756,0.448430,0.555556,0.523560,0.058187,175.006,102.483849,0.585602,1.2,1.643168,1.36930

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
13327,11:00,Rússia Superliga,Chelyabinsk,Barnaul,2.65,Back Home
13372,05:00,Austrália Liga Feminina Wnbl,Townsville F,Perth Lynx F,1.54,Back Home
13385,00:00,Eua Nba,Los Angeles Clippers,Los Angeles Lakers,1.63,Back Home
13396,13:30,Grécia A1 Feminina,Panathinaikos F,Athinaikos F,1.84,Back Home
13412,20:30,Eua Ncaa,UMass Lowell,Bryant University,1.55,Back Home
